Preprocessing of the data
From the data extraction via Wikipedia to clean the dataframe 

In [8]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")

extracted = soup.find('div', class_='mw-parser-output')
tabledata = extracted.table.tbody

column_names = ['Postalcode','Borough','Neighborhood']
TableTrimed = pd.DataFrame(columns = column_names)

for tri in tabledata.find_all('tr'):
   cnt=0
   for tdi in tri.find_all('td'):
       cnt=cnt+1
       if cnt ==1:
           Postem=tdi.text.strip('\n').replace(']','')
       elif cnt==2:
           Bortem=tdi.text.strip('\n').replace(']','')
           if Bortem.find('Not assigned') != -1:
               Bortem=np.nan
       elif cnt==3:
           Neitem=tdi.text.strip('\n').replace(']','')
           if Neitem.find('Not assigned') != -1:
               Neitem=Bortem
           TableTrimed = TableTrimed.append({'Postalcode': Postem,'Borough': Bortem,'Neighborhood': Neitem},ignore_index=True)

TableTrimedr=TableTrimed.dropna() 

print(TableTrimedr.shape)

(103, 3)


# Below is the main part for this portion of the assignment

In [11]:
Import geocoder library and create a definition that returns latitude and longitude by inputting a postal code

SyntaxError: invalid syntax (<ipython-input-11-ff5735d60b6e>, line 1)

In [ ]:
import geocoder
def returnll(Postal):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(Postal))
    lati_long_coords = g.latlng
    return lati_long_coords

From the dataframe that has been cleaned, extract the postal code information.
Then, by using the definition described above, all the combinations of latitude and longitude for the postal codes are collected.
Because the data is stored as "List", convert the type to "Dataframe" and merge the latitude and longitude dataframe to the dataframe with postalcode, borough, and neiborhood.

In [10]:
PC=TableTrimedr['Postalcode']
lllist=[returnll(PC) for PC in PC.tolist()]
lldf = pd.DataFrame(lllist, columns=['Latitude', 'Longitude'])
TableTrimedr['Latitude'] = lldf['Latitude']
TableTrimedr['Longitude'] = lldf['Longitude']

<ipython-input-10-b652fac5e386>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TableTrimedr['Latitude'] = lldf['Latitude']
<ipython-input-10-b652fac5e386>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TableTrimedr['Longitude'] = lldf['Longitude']


In [12]:
TableTrimedr.head(12)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.65512,-79.36264
3,M4A,North York,Victoria Village,43.72327,-79.45042
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66253,-79.39188
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.66263,-79.52831
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.81139,-79.19662
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.70718,-79.31192
9,M1B,Scarborough,"Malvern, Rouge",43.65739,-79.37804
11,M3B,North York,Don Mills,43.65034,-79.55362
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.78574,-79.15875
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.72168,-79.34352
